# Motor
Here are Motor and MotorBridge classes that I created to use with ROS on my RaspberryPi computer. You can control speed and rotation direction for one or two motors. To try it as is, you need PCA9685 16-Channel 12-Bit PWM Servo Driver and TB6612FNG Dual Motor Driver. Sorry, no circuit diagram yet :( 

In [17]:
#!/usr/bin/env python
import RPi.GPIO as GPIO
import time
from board import SCL, SDA
import busio
from adafruit_pca9685 import PCA9685
from adafruit_servokit import ServoKit
import math

def logdebug(msg):
    print(msg)
def loginfo(msg):
    print(msg)
def logwarn(msg):
    print(msg)
def logerr(msg):
    print(msg)
def logfatal(msg):
    print(msg)

ENC_OUTCOME  = [0, 1, -1, 0, -1, 0, 0, 1, 1, 0, 0, -1, 0, -1, 1, 0]

class Motor():
    def __init__(self, name:str, gpio_IN1:int, gpio_IN2:int, chn_PWM:int, \
                gpio_phase_A:int = math.nan, gpio_phase_B:int = math.nan, ticks_per_revolution:int = 1):
        self.__name = name
        self.__gpio_IN1 = gpio_IN1
        self.__gpio_IN2 = gpio_IN2
        self.__ticks_per_revolution = ticks_per_revolution
        self.__chn_PWM = chn_PWM

        if not math.isnan(gpio_phase_A):
            self.__gpio_encPhaseA = gpio_phase_A
        else:
            self.__gpio_encPhaseA = math.nan

        if not math.isnan(gpio_phase_B):
            self.__gpio_encPhaseB = gpio_phase_B
        else:
            self.__gpio_encPhaseB = math.nan

        self.__throttle = 0
        self.__counter = 0
        self.__prev_counter = 0
        self.__RPS = 0
        self.__prev_AB = 0b00
        self.__encoder_active = False

    def init_hw(self):
        """initialize hardware"""
        # set GPIO mode to BCM
        GPIO.setmode(GPIO.BCM)

        GPIO.setup(self.__gpio_IN1, GPIO.OUT)
        GPIO.setup(self.__gpio_IN1, GPIO.LOW)
        GPIO.setup(self.__gpio_IN2, GPIO.OUT)
        GPIO.setup(self.__gpio_IN2, GPIO.LOW)

        if not math.isnan(self.__gpio_encPhaseA) and not math.isnan(self.__gpio_encPhaseB):
            GPIO.setup(self.__gpio_encPhaseA, GPIO.IN)
            GPIO.setup(self.__gpio_encPhaseB, GPIO.IN)

            # https://sourceforge.net/p/raspberry-gpio-python/wiki/Inputs/
            GPIO.add_event_detect(self.__gpio_encPhaseA, GPIO.BOTH)
            GPIO.add_event_callback(self.__gpio_encPhaseA, self.__phase_change_callback)

            GPIO.add_event_detect(self.__gpio_encPhaseB, GPIO.BOTH)
            GPIO.add_event_callback(self.__gpio_encPhaseB, self.__phase_change_callback)

            self.__encoder_active = True

        i2c = busio.I2C(SCL, SDA)
        self.__pca = PCA9685(i2c, address = 0x40)
        # the frequency used for PWM control in the PCA9685 is adjustable from about 24Hz to 1526Hz
        self.__pca.frequency = 1526

    def __reset_encoder(self):
        self.__counter = 0
        self.__prev_counter = 0
        self.__prev_AB = 0b00

    def __phase_change_callback(self, channel):
        """Callback function for encoder"""

        # great video with an explanation of how encoders work
        # https://www.youtube.com/watch?v=p4BCFhIuC88
        A = GPIO.input(self.__gpio_encPhaseA)
        B = GPIO.input(self.__gpio_encPhaseB)
        currentAB = (A << 1) | B
        idx = (self.__prev_AB << 2) | currentAB
        self.__counter += ENC_OUTCOME[idx]
        self.__prev_AB = currentAB

    def __forward(self):
        GPIO.output(self.__gpio_IN1, GPIO.LOW)
        GPIO.output(self.__gpio_IN2, GPIO.HIGH)

    def __backward(self):
        GPIO.output(self.__gpio_IN1, GPIO.HIGH)
        GPIO.output(self.__gpio_IN2, GPIO.LOW)

    def __stop(self):
        GPIO.output(self.__gpio_IN1, GPIO.LOW)
        GPIO.output(self.__gpio_IN2, GPIO.LOW)

    def update_RPS(self):
        """Must be called once per second"""
        if not self.__encoder_active:
            return

        self.__RPS = self.__counter - self.__prev_counter
        self.__prev_counter = self.__counter

    @property
    def RPS(self):
        """Motor rate per second"""
        return self.__RPS

    @property
    def ticks_per_revolution(self):
        """Encoder ticks count per revolution"""
        return self.__ticks_per_revolution

    @property
    def name(self):
        """Motor name"""
        return self.__name

    @property
    def counter(self):
        """Current counter value"""
        return self.__counter

    @property
    def throttle(self):
        """Current throttle value"""
        return self.__throttle

    @throttle.setter
    def throttle(self, value:float):
        """Sets rotation direction and speed. Ranging from -1.0 (full speed backward) to 1.0 (full speed forward), 0.0 (stop)"""
        if value > 1.0 or value < -1.0:
            logwarn("throttle must be between -1.0 and 1.0. Passed value is: %4.2f" % value)
            return

        if value == 0: # stop motor
            self.__stop()
            self.__pca.channels[self.__chn_PWM].duty_cycle = 0x0000
        elif self.__throttle == 0: # set throttle and run
            self.__reset_encoder()
            self.__pca.channels[self.__chn_PWM].duty_cycle = int(0xFFFF * abs(value))
            if value > 0:
                self.__forward()
            else:
                self.__backward()
        elif self.__throttle < 0 and value < 0: # set throttle
            self.__pca.channels[self.__chn_PWM].duty_cycle = int(0xFFFF * abs(value))
        elif self.__throttle < 0 and value > 0: # reverse direction and set throttle
            self.__stop()
            time.sleep(1)
            self.__reset_encoder()
            self.__pca.channels[self.__chn_PWM].duty_cycle = int(0xFFFF * abs(value))
            self.__forward()
        elif self.__throttle > 0 and value > 0: # set throttle
            self.__pca.channels[self.__chn_PWM].duty_cycle = int(0xFFFF * abs(value))
        elif self.__throttle > 0 and value < 0: # reverse direction and set throttle
            self.__stop()
            time.sleep(1)
            self.__reset_encoder()
            self.__pca.channels[self.__chn_PWM].duty_cycle = int(0xFFFF * abs(value))
            self.__backward()
        
        self.__throttle = value

# left motor(M1) is inversed: AO1 GND, AO2 +
# right motor(M2): BO1 +, BO2 GND
class MotorBridge():
    def __init__(self, name:str, gpio_STBY:int):
        self.__name = name
        self.__gpio_STBY = gpio_STBY
        self.__motor1 = None
        self.__motor2 = None

    def init_hw(self):
        """initialize hardware"""
       # set GPIO mode to BCM
        GPIO.setmode(GPIO.BCM)

        GPIO.setup(self.__gpio_STBY, GPIO.OUT)
        GPIO.setup(self.__gpio_STBY, GPIO.LOW)

    @property
    def name(self):
        """Bridge name"""
        return self.__name

    def on(self):
        GPIO.output(self.__gpio_STBY, GPIO.HIGH)

    def off(self):
        GPIO.output(self.__gpio_STBY, GPIO.LOW)

    def motor_setup(self, motor1:object, motor2:object):
        if motor1 is not None:
            self.__motor1 = motor1
            loginfo("Motor 1 (%s) motor is ready to use" % self.__motor1.name)

        if motor2 is not None:
            self.__motor2 = motor2
            loginfo("Motor 2 (%s) motor is ready to use" % self.__motor2.name)

    @property
    def throttle_motor1(self):
        if self.__motor1 is not None:
            return self.__motor1.throttle
        else:
            logwarn("Motor 1 isn't active")
            return math.nan

    @property
    def throttle_motor2(self):
        if self.__motor2 is not None:
            return self.__motor2.throttle
        else:
            logwarn("Motor 2 isn't active")
            return math.nan

    @throttle_motor1.setter
    def throttle_motor1(self, value:float):
        if self.__motor1 is not None:            
            self.__motor1.throttle = value
        else:
            logwarn("Motor 1 isn't active")

    @throttle_motor2.setter
    def throttle_motor2(self, value:float):
        if self.__motor2 is not None:            
            self.__motor2.throttle = value
        else:
            logwarn("Motor 2 isn't active")

    @property
    def motor1_RPS(self):
        if self.__motor1 is not None:
            return self.__motor1.RPS
        else:
            logwarn("Motor 1 isn't active")

    @property
    def motor2_RPS(self):
        if self.__motor2 is not None:
            return self.__motor2.RPS
        else:
            logwarn("Motor 2 isn't active")

    @property
    def motor1_counter(self):
        if self.__motor1 is not None:
            return self.__motor1.counter
        else:
            logwarn("Motor 1 isn't active")

    @property
    def motor2_counter(self):
        if self.__motor2 is not None:
            return self.__motor2.counter
        else:
            logwarn("Motor 2 isn't active")

    @property
    def motor1_name(self):
        if self.__motor1 is not None:
            return self.__motor1.name
        else:
            logwarn("Motor 1 isn't active")

    @property
    def motor2_name(self):
        if self.__motor2 is not None:
            return self.__motor2.name
        else:
            logwarn("Motor 2 isn't active")

    @property
    def motor1_ticks_per_revolution(self):
        if self.__motor1 is not None:
            return self.__motor1.ticks_per_revolution
        else:
            logwarn("Motor 1 isn't active")

    @property
    def motor2_ticks_per_revolution(self):
        if self.__motor2 is not None:
            return self.__motor2.ticks_per_revolution
        else:
            logwarn("Motor 2 isn't active")
    
    def update_RPS(self):
        if self.__motor1 is not None:
            self.__motor1.update_RPS()

        if self.__motor2 is not None:
            self.__motor2.update_RPS()

To avoid any damage, set values for parameters gpio port numbers and PWM channel numbers carefully. **Current values are examples and valid for my robot only**

In [23]:
try:
    left_motor = Motor("left", 27, 22, 0, 21, 20, 510)
    left_motor.init_hw()

    right_motor = Motor("right", 5, 6, 1, 8, 7, 510)
    right_motor.init_hw()

    bridge = MotorBridge("rear", 17)
    bridge.init_hw()

    bridge.motor_setup(left_motor, right_motor)
    
    bridge.on()
    
    print("%s motor going forward" % bridge.motor1_name)
    bridge.throttle_motor1 = 0.5
    time.sleep(1)
    bridge.update_RPS()
    print("%s motor revolutions per second: %d" % (bridge.motor1_name, bridge.motor1_RPS))
    time.sleep(1)
    bridge.update_RPS()
    print("%s motor revolutions per second: %d" % (bridge.motor1_name, bridge.motor1_RPS))
    time.sleep(1)
    bridge.update_RPS()
    print("%s motor revolutions per second: %d" % (bridge.motor1_name, bridge.motor1_RPS))

    print("%s motor going backward" % bridge.motor1_name)
    bridge.throttle_motor1 = -0.5
    time.sleep(1)
    bridge.update_RPS()
    print("%s motor revolutions per second: %d" % (bridge.motor1_name, bridge.motor1_RPS))
    time.sleep(1)
    bridge.update_RPS()
    print("%s motor revolutions per second: %d" % (bridge.motor1_name, bridge.motor1_RPS))
    time.sleep(1)
    bridge.update_RPS()
    print("%s motor revolutions per second: %d" % (bridge.motor1_name, bridge.motor1_RPS))
    bridge.throttle_motor1 = 0.0
    
    time.sleep(1)
    
    print("%s motor going forward" % bridge.motor2_name)
    bridge.throttle_motor2 = 0.5
    time.sleep(1)
    bridge.update_RPS()
    print("%s motor revolutions per second: %d" % (bridge.motor2_name, bridge.motor2_RPS))
    time.sleep(1)
    bridge.update_RPS()
    print("%s motor revolutions per second: %d" % (bridge.motor2_name, bridge.motor2_RPS))
    time.sleep(1)
    bridge.update_RPS()
    print("%s motor revolutions per second: %d" % (bridge.motor2_name, bridge.motor2_RPS))
    
    print("%s motor going backward" % bridge.motor1_name)
    bridge.throttle_motor2 = -0.5
    time.sleep(1)
    bridge.update_RPS()
    print("%s motor revolutions per second: %d" % (bridge.motor2_name, bridge.motor2_RPS))
    time.sleep(1)
    bridge.update_RPS()
    print("%s motor revolutions per second: %d" % (bridge.motor2_name, bridge.motor2_RPS))
    time.sleep(1)
    bridge.update_RPS()
    print("%s motor revolutions per second: %d" % (bridge.motor2_name, bridge.motor2_RPS))
    bridge.throttle_motor2 = 0.0

    bridge.off()
finally:
    GPIO.cleanup()

Motor 1 (left) motor is ready to use
Motor 2 (right) motor is ready to use
left motor going forward
left motor revolutions per second: 1172
left motor revolutions per second: 1354
left motor revolutions per second: 1359
left motor going backward
left motor revolutions per second: -1074
left motor revolutions per second: -1240
left motor revolutions per second: -1233
right motor going forward
right motor revolutions per second: 1281
right motor revolutions per second: 1474
right motor revolutions per second: 1481
left motor going backward
right motor revolutions per second: -1293
right motor revolutions per second: -1504
right motor revolutions per second: -1513
